# Data Filtering Pipeline
## Prerequisites

In [1]:
import glob
import pandas as pd
from mmproteo.utils.utils import ensure_dir_exists
from mmproteo.utils import log
from mmproteo.utils.formats.mz import FilteringProcessor, filter_files
from mmproteo.utils.processing import ItemProcessor
import wget
from openpyxl import load_workbook
import os

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

In [3]:
logger = log.DummyLogger(verbose=True)

INFO: Printing to Stdout


## Data Import

In [4]:
pwd

'/tf/workspace/notebooks'

Code to run before running this pipeline to get/create all relevant data files:

```
cd ../datasets
mkdir PXD010000
cd PXD010000
mmproteo -p PXD010000 -e mzml,mzid --thread-count 0 download extract mz2parquet
```

In [5]:
PROJECT = "PXD010000"
DATA_PATH = f"../datasets/{PROJECT}"
DUMP_PATH = f"../dumps/{PROJECT}"
TRAINING_COLUMNS_DUMP_PATH = DUMP_PATH + "/training_columns"
MZMLID_FILES_PATH = f"{DATA_PATH}/*_mzmlid.parquet"

In [6]:
# taken from https://www.biorxiv.org/content/10.1101/428334v2.supplementary-material
SPECIES_MAPPING_FILE_DOWNLOAD_URL = 'https://www.biorxiv.org/highwire/filestream/128716/field_highwire_adjunct_files/1/428334-2.xlsx'
SPECIES_MAPPING_FILE_PATH = os.path.join(DUMP_PATH, 'file_to_species_mapping.xlsx')

In [7]:
ensure_dir_exists(TRAINING_COLUMNS_DUMP_PATH)

In [8]:
MZMLID_FILE_PATHS = glob.glob(MZMLID_FILES_PATH)
len(MZMLID_FILE_PATHS)

17

In [9]:
def download_species_mapping(download_url: str, output_file: str):
    if os.path.exists(output_file):
        print(f"skipping, because '{output_file}' already exists")
    else:
        print(wget.download(download_url, out=output_file))

def read_xlsx(file: str) -> pd.DataFrame:
    workbook = load_workbook(SPECIES_MAPPING_FILE_PATH)
    worksheet = workbook[workbook.worksheets[0].title]
    data = worksheet.values
    columns = next(data)[0:]
    df = pd.DataFrame(data=data, columns=columns)
    return df

In [10]:
def get_species_mapping(download_url: str = SPECIES_MAPPING_FILE_DOWNLOAD_URL,
                       output_file: str = SPECIES_MAPPING_FILE_PATH) -> pd.DataFrame:
    download_species_mapping(download_url=download_url, output_file=output_file)
    df = read_xlsx(file=output_file)
    df = df.drop(columns='id')
    df = df.set_index('mgf_file')
    return df

def join_species_mapping(data_df: pd.DataFrame, species_df: pd.DataFrame) -> pd.DataFrame:
    df = data_df.copy()
    
    # remove '.mzml' extension
    df['mgf_file'] = df['mzml_filename'].str.slice(stop=-5)
    df = df.join(species_df, on='mgf_file')
    
    return df

In [11]:
species_df = get_species_mapping()
species_df.head(1)

skipping, because '../dumps/PXD010000/file_to_species_mapping.xlsx' already exists


,species,num_PSM,istrain
mgf_file,,,
Biodiversity_A_cryptum_FeTSB_anaerobic_1_01Jun16_Pippin_16-03-39,Acidiphilium_cryptum_JF-5,6659,Train


In [12]:
output_columns = FilteringProcessor.get_default_output_columns()
output_columns.append('species')
output_columns.append('istrain')
output_columns

['SpectrumIdentificationItem__1__PeptideEvidenceRef__PeptideSequence',
 'mz_array',
 'intensity_array',
 'species',
 'istrain']

In [13]:
def join_species(df: pd.DataFrame) -> pd.DataFrame:
    return join_species_mapping(data_df=df, species_df=species_df)

In [14]:
output_files = filter_files(input_file_paths=MZMLID_FILE_PATHS,
                            output_path=TRAINING_COLUMNS_DUMP_PATH,
                            fdr=0.01,
                            skip_existing=False,
                            output_columns=output_columns,
                            post_processor=join_species,
                            thread_count=0,
                            logger=logger)
output_files[:1]

DEBUG: Processing items with 8 subprocesses
DEBUG: Trying to fdr-filter 17 mzmlid files
INFO: Finished filtering '../datasets/PXD010000/Biodiversity_A_cryptum_FeTSB_anaerobic_1_01Jun16_Pippin_16-03-39_mzmlid.parquet' -> '../dumps/PXD010000/training_columns/Biodiversity_A_cryptum_FeTSB_anaerobic_1_01Jun16_Pippin_16-03-39_mzmlid.parquet'
INFO: Finished filtering '../datasets/PXD010000/Biodiversity_A_tumefaciens_R2A_aerobic_2_23Nov16_Pippin_16-09-11_mzmlid.parquet' -> '../dumps/PXD010000/training_columns/Biodiversity_A_tumefaciens_R2A_aerobic_2_23Nov16_Pippin_16-09-11_mzmlid.parquet'
INFO: Finished filtering '../datasets/PXD010000/Biodiversity_A_tumefaciens_R2A_aerobic_1_23Nov16_Pippin_16-09-11_mzmlid.parquet' -> '../dumps/PXD010000/training_columns/Biodiversity_A_tumefaciens_R2A_aerobic_1_23Nov16_Pippin_16-09-11_mzmlid.parquet'
INFO: Finished filtering '../datasets/PXD010000/Biodiversity_A_faecalis_LB_aerobic_01_26Feb16_Arwen_16-01-01_mzmlid.parquet' -> '../dumps/PXD010000/training_colum

[{'input_file_path': '../datasets/PXD010000/Biodiversity_A_cryptum_FeTSB_anaerobic_1_01Jun16_Pippin_16-03-39_mzmlid.parquet',
  'output_file_path': '../dumps/PXD010000/training_columns/Biodiversity_A_cryptum_FeTSB_anaerobic_1_01Jun16_Pippin_16-03-39_mzmlid.parquet',
  'original_sequence_count': 29447,
  'NaN_decoy_count': 571,
  'above_fdr_count': 20938,
  'left_decoys': 72,
  'left_targets': 7866,
  'fdr': 0.009153318077803204,
  'removed_decoys': 72,
  'removed_by_post_processor': 0,
  'final_sequence_count': 7866}]

In [18]:
processing_result = pd.DataFrame(data=output_files)
print(f"number of processing results = {len(processing_result)}")
processing_result

number of processing results = 17


,input_file_path,output_file_path,original_sequence_count,NaN_decoy_count,above_fdr_count,left_decoys,left_targets,fdr,removed_decoys,removed_by_post_processor,final_sequence_count
0,../datasets/PXD010000/Biodiversity_A_cryptum_FeTSB_anaerobic_1_01Jun16_Pippin_16-03-39_mzmlid.parquet,../dumps/PXD010000/training_columns/Biodiversity_A_cryptum_FeTSB_anaerobic_1_01Jun16_Pippin_16-03-39_mzmlid.parquet,29447,571,20938,72,7866,0.009153,72,0,7866
1,../datasets/PXD010000/Biodiversity_A_tumefaciens_R2A_aerobic_1_23Nov16_Pippin_16-09-11_mzmlid.parquet,../dumps/PXD010000/training_columns/Biodiversity_A_tumefaciens_R2A_aerobic_1_23Nov16_Pippin_16-09-11_mzmlid.parquet,44899,14491,21273,117,9018,0.012974,117,0,9018
2,../datasets/PXD010000/Biodiversity_B_cereus_ATCC14579_LB_aerobic_2_17July16_Samwise_16-04-10_mzmlid.parquet,../dumps/PXD010000/training_columns/Biodiversity_B_cereus_ATCC14579_LB_aerobic_2_17July16_Samwise_16-04-10_mzmlid.parquet,41267,638,12571,273,27785,0.009825,273,0,27785
3,../datasets/PXD010000/Biodiversity_B_cereus_PN_L_CL_1_09Oct16_Pippin_16-05-06_mzmlid.parquet,../dumps/PXD010000/training_columns/Biodiversity_B_cereus_PN_L_CL_1_09Oct16_Pippin_16-05-06_mzmlid.parquet,40647,451,14018,247,25931,0.009525,247,0,25931
4,../datasets/PXD010000/Biodiversity_B_cereus_ATCC14579_LB_aerobic_1_17July16_Samwise_16-04-10_mzmlid.parquet,../dumps/PXD010000/training_columns/Biodiversity_B_cereus_ATCC14579_LB_aerobic_1_17July16_Samwise_16-04-10_mzmlid.parquet,41390,651,12595,279,27865,0.010013,279,0,27865
5,../datasets/PXD010000/Biodiversity_B_bifidum_CMcarb_anaerobic_03_26Feb16_Arwen_16-01-01_mzmlid.parquet,../dumps/PXD010000/training_columns/Biodiversity_B_bifidum_CMcarb_anaerobic_03_26Feb16_Arwen_16-01-01_mzmlid.parquet,36793,40,20566,128,16059,0.007971,128,0,16059
6,../datasets/PXD010000/Biodiversity_A_tumefaciens_R2A_aerobic_2_23Nov16_Pippin_16-09-11_mzmlid.parquet,../dumps/PXD010000/training_columns/Biodiversity_A_tumefaciens_R2A_aerobic_2_23Nov16_Pippin_16-09-11_mzmlid.parquet,44379,14323,21237,104,8715,0.011933,104,0,8715
7,../datasets/PXD010000/Biodiversity_A_faecalis_LB_aerobic_01_26Feb16_Arwen_16-01-01_mzmlid.parquet,../dumps/PXD010000/training_columns/Biodiversity_A_faecalis_LB_aerobic_01_26Feb16_Arwen_16-01-01_mzmlid.parquet,37414,331,19204,145,17734,0.008176,145,0,17734
8,../datasets/PXD010000/Biodiversity_B_bifidum_CMcarb_anaerobic_02_26Feb16_Arwen_16-01-01_mzmlid.parquet,../dumps/PXD010000/training_columns/Biodiversity_B_bifidum_CMcarb_anaerobic_02_26Feb16_Arwen_16-01-01_mzmlid.parquet,36746,36,20501,138,16071,0.008587,138,0,16071
9,../datasets/PXD010000/Biodiversity_B_bifidum_CMcarb_anaerobic_01_26Feb16_Arwen_16-01-01_mzmlid.parquet,../dumps/PXD010000/training_columns/Biodiversity_B_bifidum_CMcarb_anaerobic_01_26Feb16_Arwen_16-01-01_mzmlid.parquet,36680,37,20049,130,16464,0.007896,130,0,16464


In [19]:
pd.read_parquet(processing_result.output_file_path[0])

,SpectrumIdentificationItem__1__PeptideEvidenceRef__PeptideSequence,mz_array,intensity_array,species,istrain
28,KTANGKPTSAARPTPTRR,"[119.026276, 129.10242, 130.106, 136.06168, 145.0969, 156.76949, 164.85573, 170.10326, 173.0915, 175.11906, 186.12355, 192.22217, 283.13882, 354.1733, 397.3297, 437.57962, 437.91693, 438.25174, 448.74512, 455.22256, 458.75394, 474.01434, 474.27203, 495.83508, 531.6236, 548.31146, 556.8167, 557.3126, 568.8131, 569.3073, 577.3114, 577.81494, 578.3125, 584.3174, 595.3419, 598.8284, 599.3284, 600.345, 604.34985, 607.34326, 607.8346, 612.3402, 624.85504, 625.8565, 626.349, 626.85126, 633.8519, 634.3542, 646.85846, 647.3572, 647.85675, 655.8684, 656.3654, 656.8717, 739.9118, 748.41516, 748.92145, 1094.5874, 1136.6008, 1198.9551]","[392.24573, 12147.2705, 666.3889, 823.4881, 972.2865, 361.78607, 417.08383, 456.71222, 444.43277, 6938.9653, 2749.4, 362.23166, 3266.9854, 705.06445, 421.15955, 2514.085, 1735.2051, 406.9606, 445.42236, 577.852, 402.63007, 934.3149, 581.1236, 4186.4634, 817.3599, 485.8316, 1748.2828, 535.7135, 724.7904, 587.59784, 487.24197, 2499.4954, 494.83868, 487.2329, 582.5176, 702.2936, 681.7647, 558.97064, 571.6264, 3133.6794, 740.22906, 611.37427, 613.9938, 2905.3833, 2306.8975, 933.85876, 2563.092, 1895.7808, 2862.7026, 4704.8896, 2356.066, 7795.0273, 2066.8728, 536.8591, 464.84116, 488.13596, 565.3722, 663.11945, 1888.6259, 485.75937]",Acidiphilium_cryptum_JF-5,Train
67,TANGKPTSAARPTPTRR,"[100.076126, 101.07142, 102.65847, 112.08731, 116.07152, 127.05014, 127.08706, 129.10248, 130.09898, 131.08194, 131.0964, 136.06255, 141.06573, 141.10223, 144.07684, 145.09726, 146.101, 147.09102, 149.04485, 151.08597, 155.08131, 157.10779, 158.09236, 167.05525, 168.05492, 169.06157, 169.09734, 171.11298, 173.09233, 175.11914, 176.12292, 178.10922, 179.10727, 181.09666, 186.12383, 187.12572, 189.115, 196.10718, 199.10756, 208.95291, 238.11821, 248.10284, 255.14467, 255.65672, 265.12866, 266.1132, 268.12704, 282.15506, 283.13962, 284.145, 285.13333, 286.1314, 287.12854, 318.92264, 328.43604, 336.16617, 342.01868, 342.53687, 354.17944, 359.02774, 360.029, 361.02628, 364.21445, 374.71375, 377.2116, 398.96924, 399.22525, 399.47473, 399.56046, 403.22864, 403.47565, 403.72675, 403.97348, 405.22763, 405.5638, 408.55597, 412.47546, 416.72723, 416.98312, 421.2324, 421.48282, 422.90533, 423.23853, 426.24008, 430.08963, 431.57574, 431.9065, 432.23834, 437.58102, 437.91638, 438.25027, 441.7452, 441.993, 446.23978, 446.4949, 446.776, 455.2206, 474.27643, 477.7837, 478.76593, ...]","[970.92377, 8024.293, 438.97754, 2044.9241, 561.6645, 562.46136, 1025.5193, 45620.453, 2429.1597, 493.67728, 5760.535, 564.12805, 725.96246, 687.16113, 480.4321, 39380.633, 923.3975, 599.9227, 941.59937, 700.46454, 1127.3604, 981.13477, 618.1566, 3090.163, 643.3061, 551.7212, 1692.1259, 3665.422, 11353.051, 25869.855, 682.6219, 493.28824, 1065.8511, 647.7293, 11052.513, 726.6507, 550.3089, 4186.7534, 2652.8933, 2889.251, 3592.6729, 2128.6946, 842.402, 431.40222, 2923.3171, 2151.774, 488.2678, 2129.3726, 13352.417, 636.7075, 699.53015, 617.92163, 745.91907, 2386.8274, 621.1262, 1157.5568, 2568.4563, 544.1001, 858.95496, 7178.6084, 18405.688, 1932.2197, 828.2166, 608.3685, 1130.984, 1097.5002, 610.0769, 602.9149, 541.29913, 2489.858, 4889.5854, 1031.2015, 566.7529, 2333.2449, 1020.2636, 1052.1848, 648.9998, 882.9873, 708.9695, 551.6752, 640.3782, 629.0327, 565.07196, 635.3501, 986.94135, 2477.0735, 2930.9094, 731.61957, 28104.584, 11376.41, 3264.2295, 2105.532, 1697.2511, 2107.8147, 1077.1783, 2534.8892, 538.2479, 680.5165, 727.0574, 972.1554, ...]",Acidiphilium_cryptum_JF-5,Train
69,KTANGKPTSAARPTPTR,"[101.07131, 101.10792, 110.071556, 112.05059, 116.07049, 123.09159, 129.1025, 130.09755, 130.1063, 131.0815, 131.09616, 131.98347, 136.06207, 141.10103, 145.09756, 148.18243, 151.08664, 153.24802, 155.08102, 156.0769, 169.09775, 171.11278, 173.0918, 175.11919, 185.12874, 186.0875, 186.12366, 196.10757, 199.10687, 210.32182, 238.11945, 252

In [17]:
processing_result.describe()

,original_sequence_count,NaN_decoy_count,above_fdr_count,left_decoys,left_targets,fdr,removed_decoys,removed_by_post_processor,final_sequence_count
count,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.0,234.000000
mean,41519.837607,906.876068,17313.927350,221.341880,23077.692308,0.009570,221.341880,0.0,23077.692308
std,7578.393316,1978.301972,6012.774545,72.020999,7206.748798,0.000557,72.020999,0.0,7206.748798
min,28756.000000,36.000000,7036.000000,14.000000,1456.000000,0.007845,14.000000,0.0,1456.000000
25%,36696.500000,202.000000,13414.000000,172.750000,18354.250000,0.009432,172.750000,0.0,18354.250000
50%,41601.500000,348.500000,16493.000000,233.000000,24053.000000,0.009660,233.000000,0.0,24053.000000
75%,45195.750000,587.500000,20724.250000,261.500000,26863.750000,0.009825,261.500000,0.0,26863.750000
max,86594.000000,14491.000000,36596.000000,570.000000,57972.000000,0.012974,570.000000,0.0,57972.000000


In [18]:
processing_result.drop(columns=['input_file_path', 'output_file_path']).sum()

original_sequence_count      9.715642e+06
NaN_decoy_count              2.122090e+05
above_fdr_count              4.051459e+06
left_decoys                  5.179400e+04
left_targets                 5.400180e+06
fdr                          2.239291e+00
removed_decoys               5.179400e+04
removed_by_post_processor    0.000000e+00
final_sequence_count         5.400180e+06
dtype: float64

In [19]:
processing_result.to_parquet(f"{TRAINING_COLUMNS_DUMP_PATH}/processing_result.parquet")